In [22]:
import pandas as pd
data = pd.read_csv('data/gpascore.csv')
data.head()

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4


admit: 합격여부   
gre: 영어성적   
gpa: 학점   
rank: 지원한 대학원 수준

In [39]:
# gre, gpa group by rank
data.groupby('rank').mean()

,admit,gre,gpa
rank,,,
1,0.597015,630.447761,3.497612
2,0.430380,601.645570,3.344620
3,0.503876,579.844961,3.443411
4,0.394366,570.281690,3.268028


## Preprocessing(전처리)

In [23]:
# 결측치 확인
data.isnull().sum()

admit    0
gre      1
gpa      0
rank     0
dtype: int64

In [24]:
# 결측치가 하나 뿐이니 지우고 가자
data = data.dropna()
data.isnull().sum()

# 결측치를 평균으로 대체 > 이미 제거해버려서 없음
data.fillna(data.mean(), inplace=True)

In [25]:
# rank의 종류 파악
sorted(data['rank'].unique())

[1, 2, 3, 4]

In [26]:
# 영어점수 최소값 최대값 파악
data['gre'].min(), data['gre'].max()

(220.0, 800.0)

In [27]:
# gpa 최소값 최대값 파악
data['gpa'].min(), data['gpa'].max()

(2.12, 4.0)

In [28]:
# X와 Y 나누기
x = data[['gre', 'gpa', 'rank']]
y = data['admit']

In [29]:
x.count()

gre     425
gpa     425
rank    425
dtype: int64

> 입력층(=column갯수): 3개   
> 출력층(=target column): 1개 - 2진분류  
> 현재 레이어: 1개(입력층은 층으로 안 침. 출력층 1개)   
> 2진분류이므로 softmax가 아니라 sigmoid를 쓰면 됨   
> 이제 히든 레이어를 하나 만들 예정(2개 층이 될 것)   

---
### Deep Learning Model 만들기

In [30]:
from tensorflow import keras

#### Activation Function 종류 (다 해봐야 함)
- sigmoid: 0.5 기준으로 S자 그림 (0~1 사이)
- tanh(하이퍼탄젠트): 0 기준으로 S자 그림(-1 ~ 1 사이)
- relu: 0 아니면 양수(x가 늘어남에 따라 양의 무한대로 감, x가 음수면 y=0)
- softmax
- leakyRelu

In [31]:
model = keras.Sequential()

# 첫 번째 Hidden Layer 생성: 256노드(보통 2의 배수로 사용 - Hyperparameter)
# activation function: ReLU (Rectified Linear Unit)
model.add(keras.layers.Dense(256, activation='tanh', input_shape=(3,))) # 은닉층: 256노드
model.add(keras.layers.Dense(1, activation='sigmoid')) # 출력층: 1노드. 출력층은 이진 분류를 사용하므로 sigmoid를 사용

#### Optimizer: learning rate를 알맞게 조절해줌. 시작은 다 경사하강법
- adam: 경사하강시 점프가 최저점을 넘어가면 끝
- adagrad: 경사하강시 점프가 최저점을 넘어가면 다시 돌아옴
- adadelta
- rmsprop
- sgd: 경사하강법


##### Optimizer 선정도 유행을 타는데 요즘 adam을 많이 씀

### Loss: 손실함수
- binary_crossentropy: 분류 및 확률 예측에 사용
### metrics: 모델의 평가 기준
- accuracy
- loss 등

In [32]:
# 손실함수
model.compile(
  loss='binary_crossentropy', # 손실함수. sparse는 못 씀 - 이진 분류를 사용하므로 binary_crossentropy를 사용
  optimizer='adam',
  metrics=['accuracy']
)

##### 현재 데이터가 numpy가 아닌 pandas에 들어있어서 못 씀. numpy로 바꿔야 함

In [34]:
import numpy as np

# 학습하기
model.fit(np.array(x), np.array(y), epochs=500)
# verbose=0은 훈련 데이터를 출력하지 않음(지금 어느정도 진행중인지도 파악 불가)

Epoch 1/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6050 - accuracy: 0.6753
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6064 - accuracy: 0.6518
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6264 - accuracy: 0.6400
Epoch 4/500
14/14 [==============================] - 0s 4ms/step - loss: 0.6104 - accuracy: 0.6588
Epoch 5/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6205 - accuracy: 0.6635
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6262 - accuracy: 0.6306
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6242 - accuracy: 0.6376
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6150 - accuracy: 0.6541
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6161 - accuracy: 0.6424
Epoch 10/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6405 - accuracy: 0.6165
Epoch 11/

In [36]:
# 예측하기
model.predict(
  [
    [700, 3.7, 1],
    [700, 3.7, 2],
    [700, 3.7, 3],
    [700, 3.7, 4],
  ]
)

1/1 [==============================] - 0s 38ms/step


array([[0.69630617],
       [0.6637417 ],
       [0.6270772 ],
       [0.5863837 ]], dtype=float32)

In [38]:
# 예측하기
model.predict(
  [
    [400, 2.2, 1],
    [400, 2.2, 2],
    [400, 2.2, 3],
    [400, 2.2, 4],
  ]
)

1/1 [==============================] - 0s 39ms/step


array([[0.0608756 ],
       [0.05042913],
       [0.04204031],
       [0.03531504]], dtype=float32)